In [14]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [15]:
import numpy as np
import pandas as pd
from scipy import stats 
import ast

In [16]:
dataset = pd.read_csv('/content/drive/MyDrive/Deep_learning/airline-price-prediction.csv')
dataset.head()

,date,airline,ch_code,num_code,dep_time,time_taken,stop,arr_time,type,route,price
0,5/3/2022,Vistara,UK,812,9:45,10h 10m,1-stop\n\t\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t...,19:55,business,"{'source': 'Bangalore', 'destination': 'Hydera...","56,588"
1,18-03-2022,Vistara,UK,975,5:45,06h 30m,1-stop\n\t\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t...,12:15,business,"{'source': 'Delhi', 'destination': 'Hyderabad'}","59,649"
2,9/3/2022,GO FIRST,G8,7537,14:30,08h 10m,1-stop\n\t\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t...,22:40,economy,"{'source': 'Mumbai', 'destination': 'Hyderabad'}","3,916"
3,15-03-2022,GO FIRST,G8,287,10:40,09h 40m,1-stop\n\t\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t...,20:20,economy,"{'source': 'Mumbai', 'destination': 'Delhi'}","6,031"
4,22-03-2022,Vistara,UK,826,12:30,07h 25m,1-stop\n\t\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t...,19:55,economy,"{'source': 'Chennai', 'destination': 'Kolkata'}","7,081"


In [17]:
dataset['formatted_date'] = 0
dataset['source'] = 0
dataset['destination'] = 0
dataset['flight_day'] = 0
dataset['flight_month'] = 0
dataset['flight_year']=0
dataset['num_of_stops'] = 0
dataset['formatted_price'] =0
dataset['num_of_hours_taken'] =0
dataset['logarithmic_price'] =0
dataset['one_stop_in']= 'Not found'

In [18]:
def format_date(dataFrame,column,formatted_column):
    for date in range(len(dataFrame[column])):
        dataFrame[formatted_column][date] = dataFrame[column][date].replace('-','/')
    dataFrame = dataFrame.drop(columns = column)
    return dataFrame

In [19]:
dataset = format_date(dataset,'date','formatted_date')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [20]:
def split_route():
    observation = 0
    for route in dataset['route']:
       flight = ast.literal_eval(route)
       dataset['source'][observation] = flight['source']
       dataset['destination'][observation] = flight['destination']
       observation+=1

In [21]:
def split_date():
    observation = 0
    for date in dataset['formatted_date']:
       format = date.split('/')
       dataset['flight_day'][observation] = format[0]
       dataset['flight_month'][observation] =format[1]
       dataset['flight_year'][observation] =format[2]
       observation+=1

In [22]:
split_route()
split_date()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_l

In [23]:
dataset.stop.value_counts()

1-stop\n\t\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\t\t\t\t                      194814
non-stop                                                                         28944
2+-stop                                                                          10612
1-stop\n\t\t\t\t\t\t\t\t\t\t\t\tVia IXU\n\t\t\t\t\t\t\t\t\t\t\t\t                 1487
1-stop\n\t\t\t\t\t\t\t\t\t\t\t\tVia IDR\n\t\t\t\t\t\t\t\t\t\t\t\t                 1099
1-stop\n\t\t\t\t\t\t\t\t\t\t\t\tVia Patna\n\t\t\t\t\t\t\t\t\t\t\t\t                559
1-stop\n\t\t\t\t\t\t\t\t\t\t\t\tVia Indore\n\t\t\t\t\t\t\t\t\t\t\t\t               299
1-stop\n\t\t\t\t\t\t\t\t\t\t\t\tVia PAT\n\t\t\t\t\t\t\t\t\t\t\t\t                  273
1-stop\n\t\t\t\t\t\t\t\t\t\t\t\tVia MYQ\n\t\t\t\t\t\t\t\t\t\t\t\t                  267
1-stop\n\t\t\t\t\t\t\t\t\t\t\t\tVia Bhubaneswar\n\t\t\t\t\t\t\t\t\t\t\t\t          248
1-stop\n\t\t\t\t\t\t\t\t\t\t\t\tVia KLH\n\t\t\t\t\t\t\t\t\t\t\t\t                  231
1-stop\n\t\t\t\t\t\t\t\t\t\t\t\tVia JGB\n\t

In [24]:
stops = dataset['stop']
stops

0         1-stop\n\t\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t...
1         1-stop\n\t\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t...
2         1-stop\n\t\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t...
3         1-stop\n\t\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t...
4         1-stop\n\t\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t...
                                ...                        
240203    1-stop\n\t\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t...
240204    1-stop\n\t\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t...
240205                                            non-stop 
240206    1-stop\n\t\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t...
240207    1-stop\n\t\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t...
Name: stop, Length: 240208, dtype: object

In [25]:
def split_num_of_stops():
  observation= 0
  for stop in dataset['stop']:
    if stop[:8] == 'non-stop':
      dataset['num_of_stops'][observation] = 0
    elif stop[:6] == '1-stop':
      dataset['num_of_stops'][observation] = 1
    else:
      dataset['num_of_stops'][observation] = 2
    observation+=1

In [26]:
split_num_of_stops()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [28]:
def find_where_is_the_stop():
  observation = 0
  for stop in dataset['stop']:
     if stop[:6] == '1-stop':
        splitted = stop.split(' ')
        if len(splitted)>1:
           fine_splitting = splitted[1].split('\n')
           dataset['one_stop_in'][observation] = fine_splitting[0]
     else :
           dataset['one_stop_in'][observation] = 'Not found'
     observation+=1


In [29]:
dataset.one_stop_in.value_counts()

Not found    240208
Name: one_stop_in, dtype: int64

In [30]:
find_where_is_the_stop()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [31]:
dataset.one_stop_in.value_counts()

Not found         234370
IXU                 1487
IDR                 1099
Patna                559
Indore               299
PAT                  273
MYQ                  267
Bhubaneswar          248
KLH                  231
JGB                  149
JRG                  135
STV                  129
Delhi                123
BBI                  119
Hyderabad            107
IXE                   91
Ranchi                86
Raipur                78
Chennai               64
Guwahati              46
Mangalore             35
Mysore                33
Mumbai                27
VTZ                   22
Nagpur                21
Kolkata               18
NDC                   17
RPR                   16
Surat                 11
Lucknow               11
GOP                   10
Vishakhapatnam         7
NAG                    7
Kolhapur               4
GAU                    3
IXR                    3
GAY                    2
HYD                    1
Name: one_stop_in, dtype: int64

In [33]:
def fix_price_format():
  observation = 0
  for price in dataset['price']:
    number = price.split(',')
    cost = number[0] + number[1]
    dataset['formatted_price'][observation] = int(cost)
    observation+=1


In [34]:
fix_price_format()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [35]:
dataset.drop(columns = ['stop','route','flight_year','price'],inplace=True)

In [36]:
dataset.sort_values(by = ['flight_day','flight_month'],inplace =True,kind = 'quicksort',ascending = False)

In [38]:
dataset.airline.value_counts()

Vistara      102177
Air India     64754
Indigo        34571
GO FIRST      18621
AirAsia       12785
SpiceJet       7216
StarAir          48
Trujet           36
Name: airline, dtype: int64

In [39]:
dataset.ch_code.value_counts()

UK    102177
AI     64754
6E     34571
G8     18621
I5     12785
SG      7216
S5        48
2T        36
Name: ch_code, dtype: int64

In [40]:
def calculate_time_taken():
  observation = 0
  for time_taken in dataset['time_taken']:
     hours = time_taken.split(' ')
     if int(hours[1][:2]) >= 40:
        dataset['num_of_hours_taken'][observation] = int(hours[0][:2]) + 1
     else :
        dataset['num_of_hours_taken'][observation] = int(hours[0][:2]) + 1
     observation+=1

In [41]:
dataset['logarithmic_price']= np.log(dataset['formatted_price'])

In [42]:
dataset['type'] = [1 if kind_of_trip == 'business' else 0 for kind_of_trip in dataset['type']]

In [44]:
corr , pvalue = stats.pearsonr(dataset['num_code'],dataset['logarithmic_price'])
pvalue

0.0

In [45]:
corr

-0.23479129542740013